# GPT4 API

## Goal

Use GPT4 API to guess the prompts of the leaked dataset.

## Imports

In [ ]:
import os
from openai import OpenAI
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from prometeo.evaluation import get_sharpened_cosine_similarity, estimate_mean

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Code

In [ ]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'], organization=os.environ['OPENAI_API_ORG'])

In [ ]:
def chat_with_gpt4(prompt, temperature=0):
    completion = client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
    )
    return completion.choices[0].message.content

In [ ]:
prompt_template = """
Analyze the original and rewritten text and answer with the most likely text prompt that was given to rewrite or make stylistic changes to the original text.

- The text prompt should be a single sentence. Reply just with a short sentence and do not add any notes or comments.
- Sometimes the rewritten text will have hints about the text prompt. For example if it starts by
  Reworded, Rephrased, Translated, etc. you should include that word in the text prompt.
- Unless necessary do not make reference to details in the original text and keep the text prompt abstract and generic.

## Original text

{original_text}

## Rewritten text

{rewritten_text}

## Output format

Let's do the task step by step:

1. On a first step analyze the differences of the texts in less than 30 words.
2. On a second step write the most likely prompt using json format
"""

In [ ]:
def parse_prompt(response):
    if '{' not in response:
        print(f'Response does not contain json: {response}')
        return response

    json_text = '{' + response.split('{')[1].split('}')[0] + '}'
    data = eval(json_text)
    if isinstance(data, dict):
        return list(data.values())[0]
    elif isinstance(data, set):
        return data.pop()
    raise ValueError(f'Could not parse prompt from {response}')

In [ ]:
def normalize_response(response, prompt):
    normalized_response = response.split('{')[0]
    normalized_response += '{"prompt": "' + prompt + '"}'
    normalized_response = normalized_response.replace('\n\n', '\n')
    return normalized_response

## Calling to GPT4 API

- https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo
- https://platform.openai.com/docs/api-reference/chat/create?lang=python
- https://platform.openai.com/docs/api-reference/introduction

The most recent model is `gpt-4-0125-preview`

In [ ]:
df = pd.read_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/mooney_test.csv')
df.head()

In [ ]:
iterator = tqdm(zip(df['original_text'].values, df['rewritten_text'].values), total=len(df))
responses = []
for original_text, rewritten_text in iterator:
    prompt = prompt_template.format(original_text=original_text, rewritten_text=rewritten_text)
    responses.append(chat_with_gpt4(prompt))

It took 20 minutes to compute all the responses.

In [ ]:
df['gpt4_prompt'] = responses
df.to_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/mooney_test_with_gpt4.csv', index=False)

### Postprocess the responses

In [ ]:
prompts = [parse_prompt(response) for response in responses]

GPT4 works very well and I had no problem in parsing all the responses.

We might want to train later with the data, so let's normalize the responses to use always the same format.

In [ ]:
normalized_responses = [normalize_response(response, prompt) for response, prompt in zip(responses, prompts)]
df['gpt4_response'] = responses
df['gpt4_prompt'] = prompts
df['gpt4_normalized_response'] = normalized_responses
df.to_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/mooney_test_with_gpt4.csv', index=False)

Study the number of tokens of the responses.

In [ ]:
from transformers import AutoTokenizer
model_path = '/home/gbarbadillo/data/mixtral-8x7b-instruct-v0.1-hf/'
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True)

In [ ]:
df.head()

In [ ]:
df = pd.read_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/mooney_test_with_gpt4.csv')
plt.hist([len(tokenizer.tokenize(response)) for response in df.gpt4_normalized_response], bins=50);
plt.xlabel('Number of tokens')
plt.title('Distribution of GPT4 responses token length')

The mean number of tokens is around 50, and the max is below 75. Thus we should be able to generate text of that lenght on submission.

## Evaluate GPT4 on rewritten supplementary material

### Compute GPT4 responses

In [ ]:
df = pd.read_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated.csv')
df.head()

In [ ]:
iterator = tqdm(zip(df['original_text'].values, df['rewritten_text'].values), total=len(df))
responses = []
for original_text, rewritten_text in iterator:
    prompt = prompt_template.format(original_text=original_text, rewritten_text=rewritten_text)
    responses.append(chat_with_gpt4(prompt))

In [ ]:
prompts = [parse_prompt(response) for response in responses]
normalized_responses = [normalize_response(response, prompt) for response, prompt in zip(responses, prompts)]
df['gpt4_response'] = responses
df['gpt4_prompt'] = prompts
df['gpt4_normalized_response'] = normalized_responses

In [ ]:
df.to_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated_with_gpt4.csv', index=False)

One of the responses did not have json format.

### Evaluate

In [ ]:
scs = get_sharpened_cosine_similarity(df['rewrite_prompt'].values, df['gpt4_prompt'].values)
estimate_mean(scs)

In [ ]:
df['scs'] = scs
df.to_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/data/gemma_suppl_rewrite_curated_with_gpt4.csv', index=False)

GPT4 only scores 0.7036 +- 0.014 on the rewritten supplementary material. My current best score is 0.724

However if I look at GPT4 "errors" they are not errors at all:

- Many times the prompt given by GPT4 is better than the original prompt, the problem is that Gemma does not follow the given instruction
- The other big problem is that Gemma is not an [injective function](https://en.wikipedia.org/wiki/Injective_function), many different prompts can lead to the same answer. How to choose among the space of possible prompts? We could inject some
bias if we know the style of the host.

I judge that GPT4 answers are very good. I believe that a combination of GPT4 analisys and Gemma could lead to a very strong model.

What is the similarity I get if I compare my best model with GPT4?

In [ ]:
mixtral = pd.read_csv('/mnt/hdd0/Kaggle/llm_prompt_recovery/evaluations/mixtral_prompt_engineering/e1bcc54b4fcff31eeafed9bc0a0933a0a8d7b3c1955c55baf3d57c045bb433b3.csv')
print(estimate_mean(mixtral['sharpened_cosine_similarity'].values))
mixtral.head()

In [ ]:
scs = get_sharpened_cosine_similarity(mixtral['predicted_prompt'].values, df['gpt4_prompt'].values)
estimate_mean(scs)

The score rises to 0.755